In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


In [79]:
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

In [80]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [81]:
movies.shape

(35479, 3)

In [82]:
reviews.shape

(863866, 4)

In [83]:
# cell for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [84]:
movies.shape[0]

35479

In [85]:
reviews.shape[0]

863866

In [86]:
movies['genre']

0                             Documentary|Short
1                             Documentary|Short
2                             Documentary|Short
3                                           NaN
4                                  Short|Horror
                          ...                  
35474    Documentary|Adventure|Biography|Family
35475                         Documentary|Music
35476                               Documentary
35477                               Documentary
35478                                       NaN
Name: genre, Length: 35479, dtype: object

In [87]:
genres = []
for val in movies['genre']:
    try:
        genres.extend(val.split('|'))
        
    except AttributeError:
        pass

genres = set(genres)
print(len(genres))
genres

28


{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [88]:
len(np.unique(reviews.user_id))

67353

In [89]:
reviews.rating.isnull().sum()

0

In [90]:
reviews.rating.describe()

count    863866.000000
mean          7.315878
std           1.853831
min           0.000000
25%           6.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

# DATA CLEANING

In [91]:
# pull date if it exists
create_date = lambda val: val[-5:-1] if val[-1] == ')' else np.nan

# apply the function to pull the date
movies['date'] = movies['movie'].apply(create_date)
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [92]:
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
        
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['date'].apply(add_movie_year)

In [93]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [94]:
def split_genres(val):
    try:
        if val.find(g) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

In [95]:
for g in genres:
    movies[g] = movies['genre'].apply(split_genres )
    
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Fantasy,Crime,Documentary,...,Adult,News,Romance,War,Thriller,Comedy,Sport,Game-Show,Biography,Film-Noir
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
movies.iloc[1]

movie_id                                   0000010
movie          La sortie des usines Lumière (1895)
genre                            Documentary|Short
date                                          1895
1800's                                           1
1900's                                           0
2000's                                           0
Fantasy                                          0
Crime                                            0
Documentary                                      1
Drama                                            0
History                                          0
Animation                                        0
Musical                                          0
Short                                            1
Action                                           0
Music                                            0
Adventure                                        0
Horror                                           0
Talk-Show                      

In [97]:
import datetime
change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [98]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-06 02:30:50
1,2,0208092,5,1586466072,2020-04-10 02:31:12
2,2,0358273,9,1579057827,2020-01-15 08:40:27
3,2,10039344,5,1578603053,2020-01-10 02:20:53
4,2,6751668,9,1578955697,2020-01-14 04:18:17


In [99]:
def create_ranked_df(movies, reviews):
        '''
        INPUT
        movies - the movies dataframe
        reviews - the reviews dataframe
        
        OUTPUT
        ranked_movies - a dataframe with movies that are sorted by highest avg rating, more reviews, 
                        then time, and must have more than 4 ratings
        '''
        
        # Pull the average ratings and number of ratings for each movie
        movie_ratings = reviews.groupby('movie_id')['rating']
        avg_ratings = movie_ratings.mean()
        num_ratings = movie_ratings.count()
        last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
        last_rating.columns = ['last_rating']

        # Add Dates
        rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
        rating_count_df = rating_count_df.join(last_rating)

        # merge with the movies dataset
        movie_recs = movies.set_index('movie_id').join(rating_count_df)

        # sort by top avg rating and number of ratings
        ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

        # for edge cases - subset the movie list to those with only 5 or more reviews
        ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
        
        return ranked_movies
    

def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time

    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''

    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies

In [100]:

ranked_movies = create_ranked_df(movies, reviews) 

In [101]:
# Top 20 movies recommended for id 1
recs_20_for_1 = popular_recommendations('1', 20, ranked_movies)

In [102]:
recs_20_for_1

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Five Minutes (2017)',
 'Selam (2013)',
 'Let There Be Light (2017)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Chasing Happiness (2019)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Kitbull (2019)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)']

In [103]:
 num_genre_match = ranked_movies[['History','Horror']].sum(axis=1)

In [104]:
 num_genre_match

movie_id
4921860     1
5262972     0
6662050     0
2737018     0
5804314     0
           ..
10367276    0
3108604     1
3187378     0
4458206     0
7923374     0
Length: 11536, dtype: int64

In [105]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''
    REDO THIS DOC STRING
    
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Filtering movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]
        
    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
            
            
    # creating top movies list 
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies


In [106]:
# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
recs_20_for_1_filtered = popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
recs_20_for_1_filtered

["Hillary's America: The Secret History of the Democratic Party (2016)",
 'I Believe in Miracles (2015)',
 'O.J.: Made in America (2016)',
 'Ayla: The Daughter of War (2017)',
 'Hacksaw Ridge (2016)',
 'They Shall Not Grow Old (2018)',
 'Namhansanseong (2017)',
 'The Farthest (2017)',
 'Kono sekai no katasumi ni (2016)',
 'Sado (2015)',
 'Silicon Cowboys (2016)',
 '13th (2016)',
 'Ethel &amp; Ernest (2016)',
 'Paul, Apostle of Christ (2018)',
 'Kincsem (2017)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Nise - O Coração da Loucura (2015)',
 'Under sandet (2015)',
 'Only the Dead (2015)']